In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

In [3]:
df_train = pd.read_csv('data.csv', encoding="ISO-8859-1")#读入数据

In [5]:
df_all = df_train

In [6]:
df_all.shape#看一下数据形状 

(4754, 90)

In [7]:
df_all.head()  #看一下前几行长什么样子

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,¿¨ºÅ1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,¿¨ºÅ1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,¿¨ºÅ1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,¿¨ºÅ1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,¿¨ºÅ1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


#### trade_no和bank_card_no没啥价值可以删除

In [11]:
df_all = df_all.drop(['trade_no','bank_card_no'],axis=1)


KeyError: "['trade_no' 'bank_card_no'] not found in axis"

In [10]:
df_all.head()  #删除了没用的两行

,Unnamed: 0,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [18]:
type('loans_latest_day')  #目前consfin_max_limit类型是str

str

In [19]:
df_all['loans_latest_day']

0        18.0
1         2.0
2         6.0
3         4.0
4       120.0
5         4.0
6         3.0
7         NaN
8         7.0
9       142.0
10        9.0
11       35.0
12        6.0
13        9.0
14      193.0
15       27.0
16        1.0
17        7.0
18       77.0
19       37.0
20       71.0
21       23.0
22        1.0
23        9.0
24      104.0
25       82.0
26       80.0
27        6.0
28        9.0
29       34.0
        ...  
4724      5.0
4725    100.0
4726     53.0
4727     57.0
4728      9.0
4729    104.0
4730    128.0
4731     11.0
4732    250.0
4733      9.0
4734    131.0
4735      6.0
4736    156.0
4737      7.0
4738     89.0
4739      9.0
4740    163.0
4741    140.0
4742      4.0
4743     12.0
4744     88.0
4745      8.0
4746      NaN
4747     22.0
4748     54.0
4749     58.0
4750     55.0
4751     55.0
4752     16.0
4753     54.0
Name: loans_latest_day, Length: 4754, dtype: float64

In [22]:
df_all['loans_latest_day'] = df_all['loans_latest_day'].astype(float)
type(df_all['loans_latest_day'])

pandas.core.series.Series

In [32]:

print('sssssss',type(df_all['consfin_credit_limit']) )
df_all['consfin_credit_limit']

sssssss <class 'pandas.core.series.Series'>


0        1200.0
1       15100.0
2        4200.0
3       16300.0
4        8300.0
5       11200.0
6        7600.0
7           NaN
8        5500.0
9           0.0
10       9900.0
11       8300.0
12      19300.0
13       5000.0
14          0.0
15      18400.0
16      21300.0
17       4700.0
18      10100.0
19      10100.0
20       8400.0
21      16500.0
22      21900.0
23       8100.0
24      14000.0
25      10400.0
26       6500.0
27       7000.0
28      21600.0
29       3000.0
         ...   
4724     5000.0
4725    17200.0
4726     4800.0
4727     4800.0
4728    10800.0
4729     6200.0
4730    15600.0
4731    60200.0
4732     4300.0
4733     9000.0
4734     1500.0
4735    20300.0
4736    10200.0
4737    14800.0
4738     6800.0
4739     3800.0
4740    11300.0
4741    13700.0
4742     9200.0
4743     1700.0
4744     5200.0
4745     6800.0
4746        NaN
4747     4500.0
4748    17300.0
4749    49200.0
4750    10100.0
4751     7600.0
4752     7100.0
4753     4500.0
Name: consfin_credit_lim

In [34]:
type('loans_latest_day')

str

### 寻找缺失值&填充：

In [118]:
df_all.isnull().sum().sort_values(ascending = False)#找出哪些行有缺失

0                                             4754
student_feature                               2998
cross_consume_count_last_1_month               426
latest_six_month_apply                         304
apply_score                                    304
apply_credibility                              304
query_org_count                                304
query_finance_count                            304
query_cash_count                               304
query_sum_count                                304
latest_query_time                              304
latest_three_month_apply                       304
latest_one_month_apply                         304
latest_query_day                               304
loans_credibility_limit                        297
loans_count                                    297
consfin_avg_limit                              297
loans_latest_day                               297
loans_credit_limit                             297
consfin_product_count          

In [121]:
df_all = df_all.fillna(df_all.mean())   #均值填充
df_all

,Unnamed: 0,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,0
0,5,2791858,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,...,1688.000000,1200.000000,75.00000,1.000000,2.000000,1200.000000,1200.000000,12.000000,18.000000,NaN
1,10,534047,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,...,1758.000000,15100.000000,80.00000,5.000000,6.000000,22800.000000,9360.000000,4.000000,2.000000,NaN
2,12,2849787,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,...,1250.000000,4200.000000,87.00000,1.000000,1.000000,4200.000000,4200.000000,2.000000,6.000000,NaN
3,13,1809708,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,...,1541.000000,16300.000000,80.00000,5.000000,5.000000,30000.000000,12180.000000,2.000000,4.000000,NaN
4,14,2499829,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,...,1630.000000,8300.000000,79.00000,2.000000,2.000000,8400.000000,8250.000000,22.000000,120.000000,NaN
5,15,518072,0.02,0.98,2000,7.59,1.00,0.733,27.0,8.0,...,1941.000000,11200.000000,80.00000,10.000000,12.000000,20400.000000,8130.000000,3.000000,4.000000,NaN
6,16,1205125,0.02,0.98,0,23.67,0.94,0.087,10.0,54.0,...,2200.000000,7600.000000,73.00000,2.000000,2.000000,16800.000000,8900.000000,1.000000,3.000000,NaN
7,18,1129897,0.02,0.98,0,0.25,0.88,0.302,19.0,20.0,...,1820.357864,9187.009199,76.04263,4.732331,5.227507,16153.690823,8007.696881,24.112809,55.181512,NaN
8,20,2599411,0.03,0.65,0,0.31,0.76,0.472,15.0,21.0,...,4750.000000,5500.000000,79.00000,8.000000,11.000000,19200.000000,7987.000000,24.000000,7.000000,NaN
9,26,1413051,0.01,0.99,500,0.80,1.00,0.088,15.0,36.0,...,1520.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,18.000000,142.000000,NaN
